In [7]:
import os
# import sys
# sys.path.append('../../src')
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
from datetime import datetime
import time
from hydra import initialize, compose
import pathlib
# import config

import src.data_process.neg_sample as ng_sample
from src.data_process.utils import mix_merge
from src.data_process.data_split import data_split_user
from src.metrics.evaluate_ignite import CustomHR, CustomNDCG, CustomAuc_top, CustomAuc, CustomRecall_top, CustomPrecision_top
from src.model_entity import EntityCat
from src.data_utils import CatData
from src.utils.constants import DEFAULT_USER_COL,DEFAULT_ITEM_COL,DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL


from sklearn import metrics, preprocessing
from sklearn.metrics import roc_auc_score, average_precision_score, recall_score, precision_score

import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

# import argparse
torch.manual_seed(0)

In [8]:
from ignite.engine import Engine, Events, create_supervised_trainer, create_supervised_evaluator, RemovableEventHandle
from ignite.metrics import Accuracy, Loss, Metric
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.exceptions import NotComputableError
from ignite.metrics.metric import sync_all_reduce, reinit__is_reduced
from ignite.contrib.handlers.tqdm_logger import ProgressBar
# from ignite.contrib.handlers import TensorboardLogger 
from ignite.contrib.handlers.wandb_logger import *

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [10]:
with initialize(version_base=None, config_path="../src/conf"):
    cfg = compose(config_name="config", overrides=[])

In [11]:
if device.type =='cpu':
    BATCH_SIZE = cfg.params.batch_size_cpu
    EPOCHS  = cfg.params.epochs_cpu
else:
    BATCH_SIZE = cfg.params.batch_size_gpu
    EPOCHS  = cfg.params.epochs_gpu

In [20]:
if device.type == 'cpu':
    use_amp=False
    df_train_pos  = ng_sample.read_feather(pathlib.Path(cfg.path.global_temp, cfg.global_temp_data.train_pos))
    df_train_neg = pd.read_feather(pathlib.Path(cfg.path.global_temp, cfg.global_temp_data.train_neg))
#     df_test_pos = pd.read_feather(pathlib.Path(cfg.path.global_temp, cfg.global_temp_data.test_neg))
    df_test_pos = pd.read_feather(pathlib.Path(cfg.path.global_temp, cfg.global_temp_data.test_pos))
#     df_all_features = pd.read_csv(pathlib.Path(cfg.path.root, cfg.data.all_features))
#     df_train_pos = df_train_pos.sort_values(by=[DEFAULT_USER_COL]).iloc[:100,].reset_index(drop=True)
#     df_train_neg = df_train_neg.sort_values(by=[DEFAULT_USER_COL]).iloc[:100*cfg.params.neg_train,].reset_index(drop=True)
else:
    use_amp=True
    df_train_pos  = ng_sample.read_feather(pathlib.Path(cfg.path.leave_one, cfg.leave_one_data.train_pos))
    df_train_neg = pd.read_feather(pathlib.Path(cfg.path.leave_one, cfg.leave_one_data.train_neg))
    df_test_ori = pd.read_feather(pathlib.Path(cfg.path.leave_one, cfg.leave_one_data.test_pos_neg))
    df_all_features = pd.read_csv(pathlib.Path(cfg.path.root, cfg.data.all_features))
    df_train_pos = df_train_pos.sort_values(by=[DEFAULT_USER_COL]).reset_index(drop=True)
    df_train_neg = df_train_neg.sort_values(by=[DEFAULT_USER_COL]).reset_index(drop=True)

In [23]:
df_test_pos

,index,userid,timestamp,itemid,WindowID_user,Split,City,State,Country,Zip_user,...,ManagedOthers,ManagedHowMany,WindowID_job,City_job,State_job,Country_job,Zip_job,StartDate,EndDate,rating
0,558415,2366,2012-05-30 07:01:01.233,1030187,3,Train,Mechanicsville,VA,US,23116,...,No,0,3.0,Richmond,VA,US,00nan,2012-05-03 11:20:34.037,2012-06-02 23:59:59,1
1,558416,2366,2012-05-30 07:01:15.447,1655,3,Train,Mechanicsville,VA,US,23116,...,No,0,3.0,Richmond,VA,US,00nan,2012-05-01 15:13:02.020,2012-05-31 23:59:59,1
2,559145,9369,2012-06-05 18:57:41.593,638347,3,Train,Spartanburg,SC,US,29306,...,No,0,3.0,Spartanburg,SC,US,00nan,2012-05-07 16:50:32.430,2012-06-06 23:59:59,1
3,560274,19360,2012-05-30 06:21:25.180,523511,3,Train,Conway,SC,US,29527,...,No,0,3.0,Pawleys Island,SC,US,29585,2012-05-01 11:58:50.027,2012-05-31 23:59:59,1
4,560301,19500,2012-06-06 18:27:17.893,175822,3,Test,Washington,DC,US,20020,...,No,0,3.0,Chantilly,VA,US,00nan,2012-05-07 23:39:33.243,2012-06-06 23:59:59,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134148,1594557,1472089,2012-06-25 23:26:07.350,453575,7,Train,Indianapolis,IN,US,46204,...,No,0,7.0,Greenwood,IN,US,46142,2012-06-24 19:52:25.833,2012-07-23 23:59:59,1
134149,1594558,1472089,2012-06-25 23:29:44.633,1060730,7,Train,Indianapolis,IN,US,46204,...,No,0,7.0,Indianapolis,IN,US,46218,2012-06-22 21:47:16.630,2012-07-21 23:59:59,1
134150,1594559,1472089,2012-06-20 16:30:11.663,626925,7,Train,Indianapolis,IN,US,46204,...,No,0,7.0,Indianapolis,IN,US,46278,2012-06-05 16:21:33.017,2012-07-04 23:59:00,1
134151,1594560,1472089,2012-06-23 16:06:03.087,573732,7,Train,Indianapolis,IN,US,46204,...,No,0,7.0,Indianapolis,IN,US,00nan,2012-06-15 17:13:26.537,2012-07-14 23:59:59,1


In [11]:
df_train_pos[DEFAULT_RATING_COL] = 1

In [12]:
def concat_index(df1, df2):
    df2.index = df2.index//cfg.params.neg_train
    return pd.concat([df1, df2], axis=0).sort_index(kind='mregesort').reset_index(drop=True)

In [13]:
df_train_all = concat_index(df_train_pos, df_train_neg)

In [14]:
df_train_all['flag'] = 1
df_test_ori['flag'] = 0
df_all = pd.concat([df_train_all, df_test_ori], axis=0).reset_index(drop=True)

user features: 
       'WindowID_user', 'Split', 'City',
       'State', 'Country', 'Zip_user', 'DegreeType', 'Major', 'GraduationDate',
       'WorkHistoryCount', 'TotalYearsExperience', 'CurrentlyEmployed',
       'ManagedOthers', 'ManagedHowMany',
       
job features: 
       'WindowID_job', 'City_job',
       'State_job', 'Country_job', 'Zip_job', 'StartDate', 'EndDate',

### Choose the features and process data for the training

In [52]:
user_features = ['DegreeType', 'Major', 'GraduationDate']
user_features_extend = [DEFAULT_USER_COL] + user_features

item_features = []
item_features_extend =[DEFAULT_ITEM_COL] + item_features

base_features = [DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL]

In [53]:
df_mix_merge = mix_merge(df_all , df_all_features, user_features_extend, item_features_extend)

In [54]:
def _cat_encode(df_data, list_f, encoder):
    for f in list_f:
        df_data[f] = encoder.fit_transform(df_data[f].astype('category').cat.codes.values)
    return df_data

In [55]:
def _embedding_dimension(df_all_encode, features_to_train, max_dim=50):
    embedding_size = []
    features_to_em = [i for i in features_to_train if i !=DEFAULT_RATING_COL]
    for c in features_to_em:
        num_unique_values = int(df_all_encode[c].nunique())
        embed_dim = int(min(np.ceil(num_unique_values/2), max_dim))
        embedding_size.append([num_unique_values, embed_dim])  
    return embedding_size

In [56]:
def encode_data(df_mix_merge, features_to_code, features_to_train, max_dim=50):
    encoder = preprocessing.LabelEncoder()
    df_all_encode = _cat_encode(df_mix_merge, features_to_code, encoder)
    df_train = df_all_encode[df_all.flag==1]
    df_test = df_all_encode[df_all.flag==0]
    df_train = df_train[features_to_train]
    df_test = df_test[features_to_train]
    embedding_size = _embedding_dimension(df_all_encode, features_to_train, max_dim)
    return df_train, df_test, embedding_size

In [57]:
num_feature=[]
features_to_code = df_mix_merge.columns
features_to_train = [DEFAULT_USER_COL, DEFAULT_ITEM_COL]+ user_features + item_features +[DEFAULT_RATING_COL]


In [58]:
df_train,  df_test, embedding_size = encode_data(df_mix_merge, features_to_code, features_to_train, max_dim=cfg.params.emb_dim)

print(f'The size of embedding layers:{embedding_size}')

The size of embedding layers:[[89946, 50], [139292, 50], [7, 4], [14075, 50], [606, 50]]


## Run data check before training 

Check the ratio of positive and negative samples

In [59]:
assert len(df_train[df_train.rating==0])/len(df_train[df_train.rating==1]) == cfg.params.neg_train, 'wrong neg/pos ratio in training set'
assert len(df_test[df_test.rating==0])/len(df_test[df_test.rating==1]) == cfg.params.neg_test, 'wrong neg/pos ratio in test set '
#Check if all the users in test can be found in training set
assert sum(np.isin(df_test.userid.unique(), df_train.userid.unique(), assume_unique=True)) == len(df_test.userid.unique()), 'cold start'
#The the uniqueness of items between training and test. For a user, on common items between training and test dataset. 
assert df_all.shape[0] ==df_train.shape[0]+df_test.shape[0], 'wrong data concat'
assert sum(df_all.groupby(['userid']).apply(lambda x: len(x['itemid'].unique()))) == df_all.shape[0], 'train and test have overlap item'

## Creat the numpy array for training 

In [60]:
df_train_split, df_val_split = data_split_user(df_train, val_size=0.2)

np_train = df_train_split.values
np_val = df_val_split.values
np_test = df_test.values

In [61]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [62]:
train_dataset = CatData(np_train)
val_dataset = CatData(np_val)
test_dataset = CatData(np_test) 
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0,  worker_init_fn=seed_worker,generator=g)
val_loader = data.DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False, num_workers=0,  worker_init_fn=seed_worker,generator=g)
test_loader = data.DataLoader(test_dataset, batch_size=cfg.params.neg_test+1, shuffle=False, num_workers=0,worker_init_fn=seed_worker,generator=g )

In [63]:
model = EntityCat(embedding_size = embedding_size, num_numerical_cols = len(num_feature),
               output_size = 1)
model.to(device)

EntityCat(
  (all_embeddings): ModuleList(
    (0): Embedding(89946, 50)
    (1): Embedding(139292, 50)
    (2): Embedding(7, 4)
    (3): Embedding(14075, 50)
    (4): Embedding(606, 50)
  )
  (mlp_layers): Sequential(
    (0): Linear(in_features=204, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.4, inplace=False)
  )
  (predict_layer): Linear(in_features=100, out_features=1, bias=True)
)

In [64]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=cfg.params.lr)

## Model the training 

In [65]:
def output_trans_loss(output):
    return output['y_pred'], output['label']

val_metrics_train = {
    'auc': CustomAuc(),
    "loss": Loss(criterion, output_transform=output_trans_loss)
}

val_metrics_test = {
    'hr': CustomHR(),
    'ndcg': CustomNDCG(k=cfg.params.topk),
    'auc': CustomAuc(),
    'auc_top': CustomAuc_top(),
    'recall_top': CustomRecall_top(k=cfg.params.topk),
    'precision_top': CustomPrecision_top(k=cfg.params.topk),
    "loss": Loss(criterion, output_transform=output_trans_loss)
}

In [66]:
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
def train_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y = batch[0].to(device), batch[1].to(device)
    with torch.cuda.amp.autocast(enabled=use_amp):
        y_pred = model(x).reshape(1,-1).flatten()
        loss = criterion(y_pred, y.float())
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    return loss.item()

def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, label = batch[0].to(device), batch[1].to(device)
        y_pred = model(x).reshape(1,-1).flatten()
        label=label.float()
        return {'label':label, 'y_pred':y_pred}


def test_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, label = batch[0].to(device), batch[1].to(device)
        y_pred = model(x).reshape(1,-1).flatten()
        label=label.float()
        
        y_pred_top, indices = torch.topk(y_pred, engine.state.topk)
        
        y_pred_top = y_pred_top.detach().cpu().numpy()
        reco_item = torch.take(x[:,1], indices).cpu().numpy().tolist()
        pos_item = x[0,1].cpu().numpy().tolist()  # ground truth, item id
        label_top = label[indices].cpu().numpy()
        indices = indices.cpu().numpy()
        return {'pos_item':pos_item, 'reco_item':reco_item, 'y_pred_top':y_pred_top, 
                'label_top':label_top, 'label':label, 'y_pred':y_pred, 'y_indices':indices}
    
trainer = Engine(train_step)

train_evaluator = Engine(validation_step)
# train_evaluator.state_dict_user_keys.append('topk')

val_evaluator = Engine(validation_step)
# val_evaluator.state_dict_user_keys.append('topk')

test_evaluator = Engine(test_step)
test_evaluator.state_dict_user_keys.append('topk')

# @val_evaluator.on(Events.STARTED)
# def init_user_value():
#     val_evaluator.state.topk=3
    
# @train_evaluator.on(Events.STARTED)
# def init_user_value():
#     train_evaluator.state.topk=3

@train_evaluator.on(Events.STARTED)
def init_user_value():
    test_evaluator.state.topk=cfg.params.topk
    
    
# Attach metrics to the evaluators
for name, metric in val_metrics_train.items():
    metric.attach(train_evaluator, name)

for name, metric in val_metrics_train.items():
    metric.attach(val_evaluator, name)

    
for name, metric in val_metrics_test.items():
    metric.attach(test_evaluator, name)
    
# Eearly_stop 
def score_function(engine):
    val_loss = engine.state.metrics['auc']
    return val_loss

Eearly_stop_handler = EarlyStopping(patience=cfg.params.patience, score_function=score_function, trainer=trainer)
# val_evaluator.add_event_handler(Events.COMPLETED, Eearly_stop_handler)

In [67]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    train_evaluator.run(train_loader)
    metrics = train_evaluator.state.metrics
    auc = metrics['auc']
    loss = metrics['loss']
    print(f'Training Results- Epoch[{trainer.state.epoch}]  Avg loss: {loss:.2f} \
          Avg auc:{auc:.2f}')


@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    val_evaluator.run(val_loader)
    metrics = val_evaluator.state.metrics
    auc = metrics['auc']
    loss = metrics['loss']
    print(f'Validation Results- Epoch[{trainer.state.epoch}]  Avg loss: {loss:.2f} \
          Avg auc:{auc:.2f}')

    
@trainer.on(Events.COMPLETED)
def log_test_results(trainer):
    test_evaluator.run(test_loader)
    metrics = test_evaluator.state.metrics
    hr = metrics['hr']
    ndcg = metrics['ndcg']
    auc = metrics['auc']
    auc_top = metrics['auc_top']
    recall = metrics['recall_top']
    precision = metrics['precision_top']
    loss = metrics['loss']
    print(f"Test Results - Epoch[{trainer.state.epoch}]  Avg loss: {loss:.2f} \
     Avg ndcg: {ndcg:.2f}  Avg auc: {auc:.2f}  Avg auc_top: {auc_top:.2f} \
      Avg recall: {recall:.2f}  Avg precision: {precision:.2f}")

pbar = ProgressBar(persist=False)
pbar.attach(trainer)

In [68]:
# trainer.run(train_loader, max_epochs=2)

In [69]:
config_dict = dict(cfg.params)
config_dict['Features']='-'.join(user_features+item_features)
wandb_logger = WandBLogger(
    project="pytorch-jrs",
    name="-".join(user_features)+'-'+'-'.join(item_features),
    config=config_dict,
    tags=["entity", "jrs"]
)

to_save = {'model': model}
checkpoint_handler = ModelCheckpoint(
    wandb_logger.run.dir,
    n_saved=1, filename_prefix='best',
    score_name="auc",
    global_step_transform=global_step_from_engine(trainer)
)

val_evaluator.add_event_handler(Events.COMPLETED, checkpoint_handler, to_save)

    
wandb_logger.attach_output_handler(
    trainer,
    event_name=Events.ITERATION_COMPLETED,
    tag="training",
    output_transform=lambda loss: {"loss": loss}
)

wandb_logger.attach_output_handler(
    train_evaluator,
    event_name=Events.EPOCH_COMPLETED,
    tag="training",
    metric_names=['loss','auc'],
    global_step_transform=lambda *_: trainer.state.iteration,
)

wandb_logger.attach_output_handler(
    val_evaluator,
    event_name=Events.EPOCH_COMPLETED,
    tag="validation",
    metric_names=['loss',"auc"],
    global_step_transform=lambda *_: trainer.state.iteration,
)


wandb_logger.attach_output_handler(
    test_evaluator,
    event_name=Events.COMPLETED,
    tag="test",
    metric_names=['loss',"auc", 'hr', 'ndcg', 'auc_top', 'recall_top', 'precision_top'],
    global_step_transform=lambda *_: trainer.state.iteration,
)


wandb_logger.attach_opt_params_handler(
    trainer,
    event_name=Events.ITERATION_STARTED,
    optimizer=optimizer,
    param_name='lr'  # optional
)

# wandb_logger.watch(model) 

trainer.run(train_loader, max_epochs=EPOCHS)
wandb_logger.close()

[1/3]  33%|###3       [00:00<?]

Training Results- Epoch[1]  Avg loss: 0.87           Avg auc:0.80
Validation Results- Epoch[1]  Avg loss: 0.87           Avg auc:0.78


[1/3]  33%|###3       [00:00<?]

Training Results- Epoch[2]  Avg loss: 0.86           Avg auc:0.91
Validation Results- Epoch[2]  Avg loss: 0.86           Avg auc:0.88


[1/3]  33%|###3       [00:00<?]

...

[1/3]  33%|###3       [00:00<?]

Training Results- Epoch[99]  Avg loss: 0.62           Avg auc:0.98
Validation Results- Epoch[99]  Avg loss: 0.64           Avg auc:0.98


[1/3]  33%|###3       [00:00<?]

Training Results- Epoch[100]  Avg loss: 0.62           Avg auc:0.98
Validation Results- Epoch[100]  Avg loss: 0.64           Avg auc:0.98
Test Results - Epoch[100]  Avg loss: 0.74      Avg ndcg: 0.71  Avg auc: 0.96  Avg auc_top: 0.78       Avg recall: 0.92  Avg precision: 0.09


lr/group_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test/auc,▁
test/auc_top,▁
test/hr,▁
test/loss,▁
test/ndcg,▁
test/precision_top,▁
test/recall_top,▁
training/auc,▁▆▇█████████████████████████████████████
training/loss,██▇▆▅▄▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation/auc,▁▆▇▇▇▇██████████████████████████████████


## Quality evaluation 
display the recommended items for the user. Sample the recommended test users 

In [ ]:
unique_users_test = df_test[DEFAULT_USER_COL].unique()
# np.random.seed(123)
sub_users_test = np.random.choice(unique_users_test, 5, replace=False)
sub_users_test

In [ ]:
df_test_sub = df_test[df_test[DEFAULT_USER_COL].isin(sub_users_test)]
# df_test_sub

In [ ]:
np_test_sub = df_test_sub.values

In [ ]:
test_dataset_sample = CatData(np_test_sub)
test_sample_loader = data.DataLoader(test_dataset_sample, batch_size=cfg.params.neg_test+1, shuffle=False, num_workers=0,worker_init_fn=seed_worker,generator=g)

In [ ]:
class RecoList(Metric):
    '''
    The recommendation list 
    '''
    def __init__(self, output_transform=lambda x: [x['pos_item'], x['reco_item'], x['userid'], x['y_pred_top'], x['indices']], device="cpu"):
#         self._reco_dict = []
        self.item= np.array([])
        self.userid= np.array([])
        self.prob= np.array([])
        self.indices = []
#         self.cnt = 0
        super(RecoList, self).__init__(output_transform=output_transform, device=device)

    @reinit__is_reduced
    def reset(self):
        self.item= np.array([])
        self.userid= np.array([])
        self.prob= np.array([])
#         self.cnt = 0
        super(RecoList, self).reset()

    @reinit__is_reduced
    def update(self, output):
        gt_item = output[0]
        reco_item = output[1]
        userid = output[2]
        prob = output[3]
        indices = output[4]
        self.item = np.append(self.item, reco_item)
        self.userid = np.append(self.userid, userid)
        self.prob = np.append(self.prob, prob)
        self.indices.append(indices)
        
    @sync_all_reduce("_reco_dict")
    def compute(self):
        return self.indices, self.prob

In [ ]:
val_metrics_test_quanlity = {
    'recolist': RecoList()
}

In [ ]:
def test_q_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, label = batch[0].to(device), batch[1].to(device)
        y_pred = model(x).reshape(1,-1).flatten()
#         print(y_pred)
        label=label.float()
        y_pred_top, indices = torch.topk(y_pred, engine.state.topk)
        y_pred_top = y_pred_top.cpu().numpy().tolist()
        y_pred_top = y_pred_top
        reco_item = torch.take(x[:,1], indices).cpu().numpy().tolist()
        pos_item = x[0,1].cpu().numpy().tolist()  # ground truth, item id 
        userid = x[:engine.state.topk,0].cpu().numpy().tolist()
        return {'reco_item': reco_item, 'pos_item':pos_item, 'userid':userid, 'y_pred_top':y_pred_top, 'indices':indices.cpu().numpy().tolist()}

In [ ]:
test_q_evaluator=Engine(test_q_step)
for name, metric in val_metrics_test_quanlity.items():
    metric.attach(test_q_evaluator, name)

In [ ]:
test_q_evaluator.state_dict_user_keys.append('topk')

@test_q_evaluator.on(Events.STARTED)
def init_user_value():
    test_q_evaluator.state.topk=cfg.params.topk

In [ ]:
test_q_evaluator.run(test_sample_loader)

In [ ]:
reco_indices, prob =test_q_evaluator.state.metrics['recolist']

In [ ]:
reco_indices

In [ ]:
reco_indices_extend = []
for i in range(len(reco_indices)):
    reco_indices_extend += [(cfg.params.neg_test+1)*(i)+j for j in reco_indices[i]]

In [ ]:
# df_test_sub.iloc[reco]

In [ ]:
df_reco = df_test_sub.iloc[reco_indices_extend].copy()

In [ ]:
# df_reco

In [310]:
df_reco['prob'] = prob

In [171]:
job_info = pd.read_csv('../../data/jobs/jobid_title.csv')

In [172]:
job_info.rename(columns={'item_id':'itemid'}, inplace=True)

In [173]:
# job_info

In [312]:
pd.merge(df_reco, job_info, how='left', on='itemid')

,userid,itemid,rating,prob,Unnamed: 0,Title
0,1441,26715,1,0.990982,979738,Truck Driver- CDL
1,1441,126099,0,0.978732,595954,Data Analyst
2,1441,57477,0,0.016489,447313,"Sears HAS Store Manager_Clifton,NJ (JC)"
3,1441,46944,0,0.014807,726375,Training and Application Engineer
4,1441,128873,0,0.009509,990232,C-12 Avionics Technician - ROVER
5,1441,115613,0,0.004872,31673,Thin Film Process Engineer
6,1441,114149,0,0.004324,30451,Carpenter/Crater
7,1441,51852,0,0.004047,866714,Sales - Insurance - Account Executive
8,1441,98719,0,0.003649,25546,Layout Technician
9,1441,31263,0,0.003600,864560,Hair Stylist & Nail Techs Full/Part Time Sig...
